In [1]:
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from mmengine.evaluator import BaseMetric
from mmengine.model import BaseModel
from mmengine.runner import Runner
from mmengine.config import Config

import argparse
import os
import numpy as np
import math
import glob
import random
import itertools
import datetime
import time
import datetime
import sys
import scipy.io
# import argparse

from torch.autograd import Variable
from torchsummary import summary

import torch.nn as nn
import torch

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from einops import rearrange
from einops.layers.torch import Rearrange, Reduce

import matplotlib.pyplot as plt

# cfg = Config.fromfile('models/eeg_conformer_config.py')
from models.registry import MODELS
from datasets.registry import DATASETS

/home/msai/anishmad001/.conda/envs/mmcls/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# class PatchEmbedding(nn.Module):
#     def __init__(self, emb_size=40):
#         # self.patch_size = patch_size
#         super().__init__()

#         self.shallownet = nn.Sequential(
#             nn.Conv2d(1, 40, (1, 15), (1, 1)),
#             nn.Conv2d(40, 40, (64, 1), (1, 1)),
#             nn.BatchNorm2d(40),
#             nn.ELU(),
#             nn.AvgPool2d((1, 75), (1, 15)),  # pooling acts as slicing to obtain 'patch' along the time dimension as in ViT
#             nn.Dropout(0.5),
#         )

#         self.projection = nn.Sequential(
#             nn.Conv2d(40, emb_size, (1, 1), stride=(1, 1)),  # transpose, conv could enhance fiting ability slightly
#             Rearrange('b e (h) (w) -> b (h w) e'),
#         )


#     def forward(self, x: Tensor) -> Tensor:
#         b, _, _, _ = x.shape
#         x = self.shallownet(x)
#         x = self.projection(x)
#         return x


# class MultiHeadAttention(nn.Module):
#     def __init__(self, emb_size, num_heads, dropout):
#         super().__init__()
#         self.emb_size = emb_size
#         self.num_heads = num_heads
#         self.keys = nn.Linear(emb_size, emb_size)
#         self.queries = nn.Linear(emb_size, emb_size)
#         self.values = nn.Linear(emb_size, emb_size)
#         self.att_drop = nn.Dropout(dropout)
#         self.projection = nn.Linear(emb_size, emb_size)

#     def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
#         queries = rearrange(self.queries(x), "b n (h d) -> b h n d", h=self.num_heads)
#         keys = rearrange(self.keys(x), "b n (h d) -> b h n d", h=self.num_heads)
#         values = rearrange(self.values(x), "b n (h d) -> b h n d", h=self.num_heads)
#         energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)  
#         if mask is not None:
#             fill_value = torch.finfo(torch.float32).min
#             energy.mask_fill(~mask, fill_value)

#         scaling = self.emb_size ** (1 / 2)
#         att = F.softmax(energy / scaling, dim=-1)
#         att = self.att_drop(att)
#         out = torch.einsum('bhal, bhlv -> bhav ', att, values)
#         out = rearrange(out, "b h n d -> b n (h d)")
#         out = self.projection(out)
#         return out


# class ResidualAdd(nn.Module):
#     def __init__(self, fn):
#         super().__init__()
#         self.fn = fn

#     def forward(self, x, **kwargs):
#         res = x
#         x = self.fn(x, **kwargs)
#         x += res
#         return x


# class FeedForwardBlock(nn.Sequential):
#     def __init__(self, emb_size, expansion, drop_p):
#         super().__init__(
#             nn.Linear(emb_size, expansion * emb_size),
#             nn.GELU(),
#             nn.Dropout(drop_p),
#             nn.Linear(expansion * emb_size, emb_size),
#         )


# class GELU(nn.Module):
#     def forward(self, input: Tensor) -> Tensor:
#         return input*0.5*(1.0+torch.erf(input/math.sqrt(2.0)))


# class TransformerEncoderBlock(nn.Sequential):
#     def __init__(self,
#                  emb_size,
#                  num_heads=10,
#                  drop_p=0.5,
#                  forward_expansion=4,
#                  forward_drop_p=0.5):
#         super().__init__(
#             ResidualAdd(nn.Sequential(
#                 nn.LayerNorm(emb_size),
#                 MultiHeadAttention(emb_size, num_heads, drop_p),
#                 nn.Dropout(drop_p)
#             )),
#             ResidualAdd(nn.Sequential(
#                 nn.LayerNorm(emb_size),
#                 FeedForwardBlock(
#                     emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
#                 nn.Dropout(drop_p)
#             )
#             ))


# class TransformerEncoder(nn.Sequential):
#     def __init__(self, depth, emb_size):
#         super().__init__(*[TransformerEncoderBlock(emb_size) for _ in range(depth)])


# class ClassificationHead(nn.Sequential):
#     def __init__(self, emb_size, n_classes):
#         super().__init__()
        
#         # global average pooling
#         self.clshead = nn.Sequential(
#             Reduce('b n e -> b e', reduction='mean'),
#             nn.LayerNorm(emb_size),
#             nn.Linear(emb_size, n_classes)
#         )
#         self.fc = nn.Sequential(
#             # nn.Linear(44720, 2440),
#             # nn.ELU(),
#             nn.Linear(1080, 256),
#             nn.ELU(),
#             nn.Dropout(0.5),
#             nn.Linear(256, 32),
#             nn.ELU(),
#             nn.Dropout(0.3),
#             nn.Linear(32, 4)
#         )

#     def forward(self, x):
#         x = x.contiguous().view(x.size(0), -1)
#         out = self.fc(x)
#         return x, out


# class Conformer(nn.Sequential):
#     def __init__(self, emb_size=40, depth=6, n_classes=4, **kwargs):
#         super().__init__(

#             PatchEmbedding(emb_size),
#             TransformerEncoder(depth, emb_size),
#             ClassificationHead(emb_size, n_classes)
#         )

In [3]:
# @MODELS.register_module()
# class MMEEGConformer(BaseModel):
#     def __init__(self):
#         super().__init__()
#         self.model = Conformer()

#     def forward(self, trials, labels, mode):
#         Tok, Cls = self.model(trials)
#         if mode == 'loss':
#             return {'loss': F.cross_entropy(Cls, labels)}
#         elif mode == 'predict':
#             return Cls, labels

In [4]:
# cfg = Config.fromfile('configs/eeg_conformer_config.py')
cfg = Config.fromfile('configs/eegnet_config.py')
model = MODELS.build(cfg.model)
# model = MODELS.build(dict(type='MMEEGConformer'))

First blocks out shape: torch.Size([1, 16, 1, 15])


In [5]:
model

MMEEGNet(
  (data_preprocessor): BaseDataPreprocessor()
  (model): eegNet(
    (firstBlocks): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 8, kernel_size=(1, 125), stride=(1, 1), padding=(0, 62), bias=False)
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2dWithConstraint(8, 16, kernel_size=(64, 1), stride=(1, 1), groups=8, bias=False)
        (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ELU(alpha=1.0)
        (5): AvgPool2d(kernel_size=(1, 4), stride=4, padding=0)
        (6): Dropout(p=0.5, inplace=False)
      )
      (1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(1, 22), stride=(1, 1), padding=(0, 11), groups=16, bias=False)
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ELU(alpha=1.0)
        (4): AvgPool2d(kernel

In [6]:
# summary(model, (22, 288, 4))

In [7]:
class Accuracy(BaseMetric):
    def process(self, data_batch, data_samples):
        score, gt = data_samples
        self.results.append({
            'batch_size': len(gt),
            'correct': (score.argmax(dim=1) == gt).sum().cpu(),
        })

    def compute_metrics(self, results):
        total_correct = sum(item['correct'] for item in results)
        total_size = sum(item['batch_size'] for item in results)
        return dict(accuracy=100 * total_correct / total_size)

In [8]:
# class EEGDataset(Dataset):
#     def __init__(self, data_root, subs, batch_size, augment=True):
#         self.root = data_root
#         self.subs = subs
#         self.batch_size = batch_size
#         self.augment = augment
#         self.allData=None
#         self.allLabel=None
#         self.create_dataset()
    
#     def get_source_data(self, sub):
#         total_data = scipy.io.loadmat(os.path.join(self.root, '%d.mat' % sub))
#         train_data = total_data['data']
#         train_label = total_data['label']

#         train_data = np.transpose(train_data, (2, 1, 0))
#         train_data = np.expand_dims(train_data, axis=1)
#         train_label = np.transpose(train_label)

#         allData = train_data
#         allLabel = train_label[0]

#         shuffle_num = np.random.permutation(len(allData))
#         allData = allData[shuffle_num, :, :, :]
#         allLabel = allLabel[shuffle_num]

#         # standardize
#         target_mean = np.mean(allData)
#         target_std = np.std(allData)
#         allData = (allData - target_mean) / target_std

#         # data shape: (trial, conv channel, electrode channel, time samples)
#         return allData, allLabel

#     def create_dataset(self):
#         for sub in self.subs:
#             allData, allLabel = self.get_source_data(sub)
#             if(self.allData is None):
#                 self.allData = allData
#                 self.allLabel = allLabel
#             else:
#                 self.allData = np.concatenate((self.allData, allData), axis=0)
#                 self.allLabel = np.concatenate((self.allLabel, allLabel), axis=0)
        
#         self.allData = torch.from_numpy(self.allData)
#         self.allLabel = torch.from_numpy(self.allLabel)
        
#         if(self.augment):
#             aug_data, aug_label = self.interaug(self.allData, self.allLabel)
#             self.allData = torch.cat((self.allData, aug_data))
#             self.allLabel = torch.cat((self.allLabel, aug_label))

#     def __len__(self):
#         return len(self.allData)
    
#     def interaug(self, timg, label):  
#         aug_data = []
#         aug_label = []
#         for cls4aug in range(4):
#             cls_idx = np.where(label == cls4aug)
#             tmp_data = timg[cls_idx]
#             tmp_label = label[cls_idx]

#             tmp_aug_data = np.zeros((int(self.batch_size / 4), 1, 64, 480))
#             for ri in range(int(self.batch_size / 4)):
#                 for rj in range(8):
#                     rand_idx = np.random.randint(0, tmp_data.shape[0], 8)
#                     tmp_aug_data[ri, :, :, rj * 60:(rj + 1) * 60] = tmp_data[rand_idx[rj], :, :,
#                                                                       rj * 60:(rj + 1) * 60]

#             aug_data.append(tmp_aug_data)
#             aug_label.append(tmp_label[:int(self.batch_size / 4)])
        
#         aug_data = np.concatenate(aug_data)
#         aug_label = np.concatenate(aug_label)
#         aug_shuffle = np.random.permutation(len(aug_data))
#         aug_data = aug_data[aug_shuffle, :, :]
#         aug_label = aug_label[aug_shuffle]

#         aug_data = torch.from_numpy(aug_data)
#         aug_data = aug_data.float()
#         aug_label = torch.from_numpy(aug_label)
#         aug_label = aug_label.long()
#         return aug_data, aug_label

#     def __getitem__(self, idx):
#         return self.allData[idx].type(torch.FloatTensor), self.allLabel[idx]

In [9]:
# DATA_PATH = '/home/msai/anishmad001/codes/EEG-Conformer/data/phisionet'
# TRAIN_SUBS = [1,2,3]
# VAL_SUBS = [4,5,6]
# BATCH_SIZE = 1

In [10]:
dataset = DATASETS.build(cfg.dataset)

In [11]:
val_dataset = DATASETS.build(cfg.val_dataset)

In [12]:
dataset

In [13]:
# dataset = EEGDataset(data_root=DATA_PATH, subs=TRAIN_SUBS, batch_size=BATCH_SIZE, augment=False)

# val_dataset = EEGDataset(data_root=DATA_PATH, subs=VAL_SUBS, batch_size=BATCH_SIZE, augment=False)

In [14]:
dataset[0][0].shape

torch.Size([1, 64, 480])

In [15]:
train_dataloader = DataLoader(batch_size=32,
                              shuffle=True,
                              dataset=dataset)



In [16]:
val_dataloader = DataLoader(batch_size=32,
                            shuffle=False,
                            dataset=val_dataset)



In [17]:
runner = Runner(
    model=model,
    work_dir='./work_dir',
    train_dataloader=train_dataloader,
    optim_wrapper=dict(optimizer=dict(type=SGD, lr=0.001, momentum=0.9)),
    train_cfg=dict(by_epoch=True, max_epochs=5, val_interval=1),
    val_dataloader=val_dataloader,
    val_cfg=dict(),
    val_evaluator=dict(type=Accuracy),
)
runner.train()

04/03 02:40:31 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 332588215
    GPU 0: NVIDIA A40
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (GCC) 11.2.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_

/home/msai/anishmad001/.conda/envs/mmcls/lib/python3.10/site-packages/mmengine-0.6.0-py3.10.egg/mmengine/runner/loops.py:56: UserWarning: Dataset EEGDataset has no metainfo. ``dataset_meta`` in visualizer will be None.
  warnings.warn(
/home/msai/anishmad001/.conda/envs/mmcls/lib/python3.10/site-packages/mmengine-0.6.0-py3.10.egg/mmengine/evaluator/metric.py:47: UserWarning: The prefix is not set in metric class Accuracy.
  warnings.warn('The prefix is not set in metric class '
/home/msai/anishmad001/.conda/envs/mmcls/lib/python3.10/site-packages/mmengine-0.6.0-py3.10.egg/mmengine/runner/loops.py:335: UserWarning: Dataset EEGDataset has no metainfo. ``dataset_meta`` in evaluator, metric and visualizer will be None.
  warnings.warn(


04/03 02:40:34 - mmengine - INFO - Exp name: 20230403_024031
04/03 02:40:34 - mmengine - INFO - Saving checkpoint at 1 epochs
04/03 02:40:34 - mmengine - WARNING - `save_param_scheduler` is True but `self.param_schedulers` is None, so skip saving parameter schedulers
04/03 02:40:34 - mmengine - INFO - Epoch(val) [1][8/8]  accuracy: 22.4900
04/03 02:40:34 - mmengine - INFO - Exp name: 20230403_024031
04/03 02:40:34 - mmengine - INFO - Saving checkpoint at 2 epochs
04/03 02:40:34 - mmengine - WARNING - `save_param_scheduler` is True but `self.param_schedulers` is None, so skip saving parameter schedulers
04/03 02:40:34 - mmengine - INFO - Epoch(val) [2][8/8]  accuracy: 24.4980
04/03 02:40:35 - mmengine - INFO - Exp name: 20230403_024031
04/03 02:40:35 - mmengine - INFO - Saving checkpoint at 3 epochs
04/03 02:40:35 - mmengine - WARNING - `save_param_scheduler` is True but `self.param_schedulers` is None, so skip saving parameter schedulers
04/03 02:40:35 - mmengine - INFO - Epoch(val) [3

MMEEGNet(
  (data_preprocessor): BaseDataPreprocessor()
  (model): eegNet(
    (firstBlocks): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 8, kernel_size=(1, 125), stride=(1, 1), padding=(0, 62), bias=False)
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2dWithConstraint(8, 16, kernel_size=(64, 1), stride=(1, 1), groups=8, bias=False)
        (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ELU(alpha=1.0)
        (5): AvgPool2d(kernel_size=(1, 4), stride=4, padding=0)
        (6): Dropout(p=0.5, inplace=False)
      )
      (1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(1, 22), stride=(1, 1), padding=(0, 11), groups=16, bias=False)
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ELU(alpha=1.0)
        (4): AvgPool2d(kernel